### Source notebook is loaded

All the libraries and formulas needed for running the MTL-GARCH model are loaded. Please install all the libraries before running the code.

In [1]:
import warnings
import pandas as pd
import numpy as np
# python -m pip install yahoo-finance; pip install yahoo-finance; pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
# pip install --upgrade ta; pip install ta
import ta as ta
import sklearn as sk
from sklearn import preprocessing
from scipy.stats import t
import tensorflow as tf
from datetime import date, datetime, timedelta
from arch import arch_model
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import math

import warnings

warnings.filterwarnings(('ignore'))

In [2]:
%run Source/5_MTL_GARCH.ipynb

### MTL-GARH model

Model run for obtaining the test error of the MTL-GARCH model with the optimum dropout level obtained with the train set. The forecasts and VaR values of the different models are saved in a csv file.

In [3]:
#Index of end dates, database for validation and dataframe to collect the results are created. Model variables are defined.
Start='2018-01-01'; End='2023-06-30'; IndexEndDays=yf.download("^GSPC",start=Start,  end=End, progress=False).index
Lag=1; LagSD=5; Timestep=10; Dropout=0.10; LearningRate=0.01; Epochs=100; Alpha=0.005; DF=3.28
DataValidation = DatabaseGeneration(yf.download("^GSPC",start='2000-01-01', end=date.today()+timedelta(days=1), progress=False), Lag, LagSD)
ResultsCollection=pd.DataFrame({'Date_Forecast': [], 'Forecast_T_ANN_ARCH': [],'Forecast_GARCH':[],'Forecast_GJR_GARCH':[], 'Forecast_TARCH':[],'Forecast_EGARCH':[],'Forecast_AVGARCH':[],'Forecast_FIGARCH':[],'ReturnForecast':[],'TrueSD':[], 'VaR_T_ANN_ARCH':[], 'VaR_GARCH':[], 'VaR_GJR_GARCH':[], 'VaR_TARCH':[], 'VaR_EGARCH':[], 'VaR_AVGARCH':[], 'VaR_FIGARCH':[]})
#Loop for generating the results
for i in tqdm(range(IndexEndDays.shape[0])):
    #Database is downloaded from yahoo finance and lag of returns defined
    Database=yf.download("^GSPC",start=IndexEndDays[i].date()-timedelta(days=650), end=IndexEndDays[i].date(), progress=False)
    #Database for fitting the models is generated
    Data = DatabaseGeneration (Database, Lag, LagSD)
    #Fitting of Transformed ANN-ARCH model, ARCH models and forecasting of the next volatility value
    T_ANN_ARCH_Model = T_ANN_ARCH_Fit (Data, Lag, LagSD, Timestep, Dropout, LearningRate, Epochs, Alpha, DF)
    #VaR of ARCH models is computed
    VaR_ARCH_Models=VaR_AR_Total(Alpha, T_ANN_ARCH_Model['GARCH_fit'], T_ANN_ARCH_Model['GJR_GARCH_fit'], T_ANN_ARCH_Model['TARCH_fit'], T_ANN_ARCH_Model['EGARCH_fit'], T_ANN_ARCH_Model['AVGARCH_fit'], T_ANN_ARCH_Model['FIGARCH_fit'],T_ANN_ARCH_Model['GARCH'], T_ANN_ARCH_Model['GJR_GARCH'], T_ANN_ARCH_Model['TARCH'], T_ANN_ARCH_Model['EGARCH'], T_ANN_ARCH_Model['AVGARCH'], T_ANN_ARCH_Model['FIGARCH'])
    #Results are collected
    IterResults={'Date_Forecast': T_ANN_ARCH_Model['Date_Forecast'].date(), 'Forecast_T_ANN_ARCH': T_ANN_ARCH_Model['Forecast_T_ANN_ARCH'],'Forecast_GARCH':T_ANN_ARCH_Model['Forecast_GARCH']/100,'Forecast_GJR_GARCH':T_ANN_ARCH_Model['Forecast_GJR_GARCH']/100, 'Forecast_TARCH':T_ANN_ARCH_Model['Forecast_TARCH']/100,'Forecast_EGARCH':T_ANN_ARCH_Model['Forecast_EGARCH']/100,'Forecast_AVGARCH':T_ANN_ARCH_Model['Forecast_AVGARCH']/100,'Forecast_FIGARCH':T_ANN_ARCH_Model['Forecast_FIGARCH']/100,'ReturnForecast':T_ANN_ARCH_Model['ReturnForecast'],'TrueSD':DataValidation[DataValidation.index==pd.to_datetime(T_ANN_ARCH_Model['Date_Forecast'].date())]['TrueSD'][0], 'VaR_T_ANN_ARCH': T_ANN_ARCH_Model['VaR'], 'VaR_GARCH':VaR_ARCH_Models['VaR_GARCH'][0][0], 'VaR_GJR_GARCH':VaR_ARCH_Models['VaR_GJR_GARCH'][0][0], 'VaR_TARCH':VaR_ARCH_Models['VaR_TARCH'][0][0], 'VaR_EGARCH':VaR_ARCH_Models['VaR_EGARCH'][0][0], 'VaR_AVGARCH':VaR_ARCH_Models['VaR_AVGARCH'][0][0], 'VaR_FIGARCH':VaR_ARCH_Models['VaR_FIGARCH'][0][0]}
    ResultsCollection=ResultsCollection.append(IterResults, ignore_index=True)
    #Results are saved
    ResultsCollection.to_csv('Results/5_MTL_GARCH_test.csv',index=False)

  0%|          | 0/1382 [00:00<?, ?it/s]

14/14 [==============================] - 1s 3ms/step


  0%|          | 1/1382 [00:13<5:21:33, 13.97s/it]

14/14 [==============================] - 1s 2ms/step


  0%|          | 2/1382 [00:27<5:15:07, 13.70s/it]

...

100%|█████████▉| 1380/1382 [6:04:16<00:40, 20.31s/it]

14/14 [==============================] - 1s 3ms/step


100%|█████████▉| 1381/1382 [6:04:36<00:20, 20.29s/it]

14/14 [==============================] - 1s 2ms/step


100%|██████████| 1382/1382 [6:04:57<00:00, 15.84s/it]
